In [92]:
import polars as pl
pl.Config.set_tbl_rows(40)

df = pl.read_csv('temp.csv')
df = df.with_columns(
    [(pl.col(col) * pl.col('S')).round(1).alias(col+'_usd') for col in ['bid_price','ask_price','bid_price_P','ask_price_P']]
)

In [93]:
df.tail(5)

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.0003,0.0005,126000,0.141672,0.144328,"""5SEP25""",110106.62,0.018721,110271.67,33.0,55.1,15599.0,15891.5
"""2025-08-29T12:00:00.000000""",0.0002,0.0004,128000,0.159836,0.162493,"""5SEP25""",110106.62,0.018721,110271.67,22.0,44.0,17599.0,17891.5
"""2025-08-29T12:00:00.000000""",0.0002,0.0004,130000,0.178,0.18,"""5SEP25""",110106.62,0.018721,110271.67,22.0,44.0,19599.0,19819.2
"""2025-08-29T12:00:00.000000""",0.0001,0.0003,135000,0.223089,0.225411,"""5SEP25""",110106.62,0.018721,110271.67,11.0,33.0,24563.6,24819.2
"""2025-08-29T12:00:00.000000""",0.0001,0.0002,140000,0.2685,0.270821,"""5SEP25""",110106.62,0.018721,110271.67,11.0,22.0,29563.6,29819.2


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.special import ndtr

N = ndtr  # Normal CDF function
# N = norm.cdf

def black76_call(F, K, T, r, vol):
    """
    Black-76 call option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price 
    T: Time to expiration (years)
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (F * N(d1) - K * N(d2))

def black76_put(F, K, T, r, vol):
    """
    Black-76 put option price for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years) 
    r: Risk-free rate (for discounting)
    vol: Volatility
    """
    d1 = (np.log(F/K) + 0.5*vol**2*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return np.exp(-r * T) * (K * N(-d2) - F * N(-d1))

def black76_vega(F, K, T, r, sigma):
    """
    Black-76 vega (sensitivity to volatility) for forward contracts
    
    Parameters:
    F: Forward price
    K: Strike price
    T: Time to expiration (years)
    r: Risk-free rate  
    sigma: Volatility
    """
    d1 = (np.log(F / K) + 0.5 * sigma ** 2 * T) / (sigma * np.sqrt(T))
    return np.exp(-r * T) * F * norm.pdf(d1) * np.sqrt(T)

# Keep aliases for backward compatibility
bs_call = black76_call
bs_put = black76_put
bs_vega = black76_vega

In [ ]:
def find_vol(target_value, F, K, T, r, **kwargs):
    """
    Vectorized implied volatility solver using Newton-Raphson method with Black-76 model.
    
    All arguments can be numpy arrays, scalars, or Polars expressions.
    Returns array of implied volatilities or scalar if all inputs are scalar.
    
    Parameters:
    -----------
    target_value : array-like or scalar
        Option market prices
    F : array-like or scalar  
        Forward prices
    K : array-like or scalar
        Strike prices
    T : array-like or scalar
        Time to expiration (in years)
    r : array-like or scalar
        Risk-free interest rate (for discounting)
    option_type : str, optional
        'C' for call (default), 'P' for put
    
    Returns:
    --------
    numpy.ndarray or float
        Implied volatilities
    """
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    
    # Convert all inputs to numpy arrays for vectorization
    target_value = np.asarray(target_value, dtype=float)
    F = np.asarray(F, dtype=float)
    K = np.asarray(K, dtype=float)
    T = np.asarray(T, dtype=float)
    r = np.asarray(r, dtype=float)
    option_type = kwargs.get('option_type', 'C')
    
    # Get the broadcasted shape
    try:
        shape = np.broadcast(target_value, F, K, T, r).shape
    except ValueError as e:
        raise ValueError(f"Input arrays could not be broadcast together: {e}")
    
    # Initialize volatility array
    sigmas = np.full(shape, 0.5, dtype=float)
    done = np.zeros(shape, dtype=bool)
    
    # Newton-Raphson iterations
    for iteration in range(MAX_ITERATIONS):
        # Calculate option prices and vegas using Black-76
        if option_type == 'P':
            prices = black76_put(F, K, T, r, sigmas)
        else:
            prices = black76_call(F, K, T, r, sigmas)
        
        vegas = black76_vega(F, K, T, r, sigmas)
        
        # Calculate price differences
        diff = target_value - prices
        
        # Check convergence
        converged = np.abs(diff) < PRECISION
        
        # Only update where not converged and vega is not zero
        update_mask = ~done & (vegas != 0) & np.isfinite(vegas) & np.isfinite(diff)
        
        # Newton-Raphson update
        sigmas[update_mask] += diff[update_mask] / vegas[update_mask]
        
        # Ensure volatility stays positive
        sigmas = np.maximum(sigmas, 1e-6)
        
        # Mark converged elements as done
        done = done | converged
        
        # Early exit if all converged
        if np.all(done):
            break
    
    # Return scalar if input was scalar, otherwise return array
    if sigmas.shape == ():
        return float(sigmas)
    else:
        return sigmas

In [96]:
# Test vectorized find_vol function
import numpy as np

print("=== Testing Vectorized find_vol ===")

# Test 1: Scalar input (should work as before)
# scalar_vol = find_vol(10.5, 100.0, 105.0, 0.25, 0.05, option_type='C')
# print(f"Scalar test: {scalar_vol:.4f}")

# Test 2: Array input
prices = np.array([8.5, 10.5, 12.8])
S_vals = np.array([100.0, 100.0, 100.0])
K_vals = np.array([105.0, 105.0, 105.0])
T_vals = np.array([0.25, 0.5, 0.75])
r_vals = 0.05

vectorized_vols = find_vol(prices, S_vals, K_vals, T_vals, r_vals, option_type='C')
print(f"Vectorized test: {vectorized_vols}")

# Test 3: Mixed broadcasting (scalar and array)
mixed_vols = find_vol([8.5, 10.5], 100.0, [105.0, 110.0], 0.25, 0.05, option_type='C')
print(f"Mixed broadcasting: {mixed_vols}")

print("\n=== Ready for Polars integration ===")
print("The function now works with numpy arrays and can be used with pl.map_elements() or pl.map_batches()")

=== Testing Vectorized find_vol ===
Vectorized test: [0.50553489 0.41001943 0.38630858]
Mixed broadcasting: [0.50553489 0.69626799]

=== Ready for Polars integration ===
The function now works with numpy arrays and can be used with pl.map_elements() or pl.map_batches()


In [100]:
df_vola =\
df.with_columns(
    call_bid_vola = find_vol(
        target_value=df['bid_price_usd'].to_numpy(),
        S=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=(interest_rate:=0.0),
        option_type='C'
    ).round(4),
    call_ask_vola = find_vol(
        target_value=df['ask_price_usd'].to_numpy(),
        S=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(), 
        r=interest_rate,
        option_type='C'
    ).round(4),
    put_bid_vola = find_vol(
        target_value=df['bid_price_P_usd'].to_numpy(),
        S=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4),
    put_ask_vola = find_vol(
        target_value=df['ask_price_P_usd'].to_numpy(),
        S=df['F'].to_numpy(),
        K=df['strike'].to_numpy(),
        T=df['tau'].to_numpy(),
        r=interest_rate,
        option_type='P'
    ).round(4)
)

In [101]:
df_vola

timestamp,bid_price,ask_price,strike,bid_price_P,ask_price_P,expiry,S,tau,F,bid_price_usd,ask_price_usd,bid_price_P_usd,ask_price_P_usd,call_bid_vola,call_ask_vola,put_bid_vola,put_ask_vola
str,f64,f64,i64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2025-08-29T12:00:00.000000""",0.248,0.284217,80000,0.0001,0.0003,"""5SEP25""",110106.62,0.018721,110271.67,27306.4,31294.2,11.0,33.0,0.0,11976.9118,0.0,0.0
"""2025-08-29T12:00:00.000000""",0.202589,0.238806,85000,0.0002,0.0004,"""5SEP25""",110106.62,0.018721,110271.67,22306.4,26294.2,22.0,44.0,0.0,0.0,0.7686,0.0
"""2025-08-29T12:00:00.000000""",0.157179,0.193396,90000,0.0004,0.0006,"""5SEP25""",110106.62,0.018721,110271.67,17306.4,21294.2,44.0,66.1,0.0,0.0,0.6729,0.7132
"""2025-08-29T12:00:00.000000""",0.111768,0.147985,95000,0.0006,0.0008,"""5SEP25""",110106.62,0.018721,110271.67,12306.4,16294.2,66.1,88.1,0.0,1.0138,0.5465,0.5727
"""2025-08-29T12:00:00.000000""",0.066358,0.102575,100000,0.0014,0.0016,"""5SEP25""",110106.62,0.018721,110271.67,7306.4,11294.2,154.1,176.2,0.0,0.7614,0.4497,0.4629
"""2025-08-29T12:00:00.000000""",0.048194,0.084411,102000,0.0022,0.0025,"""5SEP25""",110106.62,0.018721,110271.67,5306.4,9294.2,242.2,275.3,0.0,0.6588,0.4186,0.4326
"""2025-08-29T12:00:00.000000""",0.0375,0.057164,105000,0.005,0.0055,"""5SEP25""",110106.62,0.018721,110271.67,4129.0,6294.2,550.5,605.6,0.0,0.4999,0.3881,0.4023
"""2025-08-29T12:00:00.000000""",0.0375,0.048082,106000,0.006,0.007,"""5SEP25""",110106.62,0.018721,110271.67,4129.0,5294.2,660.6,770.7,0.0,0.4446,0.3653,0.3903
"""2025-08-29T12:00:00.000000""",0.0375,0.039,107000,0.008,0.0085,"""5SEP25""",110106.62,0.018721,110271.67,4129.0,4294.2,880.9,935.9,0.3542,0.3875,0.359,0.3702


In [102]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_bid_vola'], mode='markers', name='Call Bid Vola', marker=dict(symbol='triangle-up', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['call_ask_vola'], mode='markers', name='Call Ask Vola', marker=dict(symbol='triangle-down', color='blue')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_bid_vola'], mode='markers', name='Put Bid Vola', marker=dict(symbol='triangle-up', color='red')))
fig.add_trace(go.Scatter(x=df_vola['strike'], y=df_vola['put_ask_vola'], mode='markers', name='Put Ask Vola', marker=dict(symbol='triangle-down', color='red')))
fig.add_vline(x=df_vola['S'][0], line_color="black", annotation_text="Spot (S)", annotation_position="top")
fig.update_layout(
    title='Implied Volatility vs Strike Price',
    xaxis_title='Strike Price',
    yaxis_title='Implied Volatility',
    yaxis=dict(range=[0.2, 0.8])  # Set the y-axis range as needed
)
fig.show()